In [2]:
from google.colab import userdata
key = userdata.get('GITHUB_TOKEN_KEY')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Internship Tasks

/content/drive/MyDrive/Internship Tasks


In [5]:
!git clone https://github.com/Zayaad-Wajid/Question-Answering-with-Transformers.git

Cloning into 'Question-Answering-with-Transformers'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [6]:
!git config --global user.email "zayaadw@example.com"
!git config --global user.name "Zayaad-Wajid"

In [7]:
%cd /content/drive/MyDrive/Internship Tasks/3-Question_

[Errno 2] No such file or directory: '/content/drive/MyDrive/Internship Tasks/Named-Entity-Recognition-NER-from-News-Articles'
/content/drive/MyDrive/Internship Tasks


In [1]:
from google.colab import files
uploaded = files.upload()

Saving dev-v1.1.json to dev-v1.1.json
Saving train-v1.1.json to train-v1.1.json


In [ ]:
!pip install transformers datasets evaluate
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
import json
import torch
import evaluate
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

In [ ]:
with open("train-v1.1.json", "r") as f:
  squad_data = json.load(f)

In [ ]:
examples = []
for article in squad_data["data"]:
  for paragraph in article["paragraphs"]:
    context = paragraph["context"]
    for qa in paragraph["qas"]:
      question = qa["question"]
      id = qa["id"]
      if qa["answers"]:
        answer_texts = [a["text"] for a in qa["answers"]]
        answer_starts = [a["answer_start"] for a in qa["answers"]]
        examples.append({
            "id": id,
            "context": context,
            "question": question,
            "answers": {
                "text": answer_texts,
                "answer_start": answer_starts
            }
        })

examples = examples[:1000]
print(examples)

[{'id': '5733be284776f41900661182', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}, {'id': '5733be284776f4190066117f', 'context': 'Architecturally, the school has a Catholic

# Loading Pre-trained QA Pipeline

In [ ]:
model = "distilbert-base-uncased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForQuestionAnswering.from_pretrained(model)

qa_pipleline = pipeline("question-answering", model = model, tokenizer = tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
predictions = []
references = []

for example in tqdm(examples):
  question = example["question"]
  context = example["context"]
  result = qa_pipleline(question = question, context = context)

  predictions.append({
      "id": example["id"],
      "prediction_text": result["answer"]
  })

  references.append({
      "id": example["id"],
      "answers": {
          "text":example["answers"]["text"],
          "answer_start": example["answers"]["answer_start"]
      }
  })

100%|██████████| 1000/1000 [00:09<00:00, 102.96it/s]


In [ ]:
squad_metric = evaluate.load("squad")

results = squad_metric.compute(predictions = predictions, references = references)

print(f"Exact Match (EM): {results['exact_match']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

Exact Match (EM): 86.10
F1 Score: 93.99


#Bonus Tasks

# Comparing BERT and ALBERT

In [ ]:
models = {
    "BERT": "bert-large-uncased-whole-word-masking-finetuned-squad",
    "ALBERT": "twmkn9/albert-base-v2-squad2"
}


In [ ]:
def evaluate_model(model_name, examples):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    qa_pipeline_model = pipeline("question-answering", model=model, tokenizer=tokenizer)

    predictions = []
    references = []

    for example in tqdm(examples):
        question = example["question"]
        context = example["context"]
        result = qa_pipeline_model(question=question, context=context)

        predictions.append({
            "id": example["id"],
            "prediction_text": result["answer"]
        })

        references.append({
            "id": example["id"],
            "answers": {
                "text": example["answers"]["text"],
                "answer_start": example["answers"]["answer_start"]
            }
        })

    squad_metric = evaluate.load("squad")
    results = squad_metric.compute(predictions=predictions, references=references)
    return results


In [ ]:
results_dict = {}

for name, model_id in models.items():
    print(f"Evaluating {name}...")
    results = evaluate_model(model_id, examples)
    results_dict[name] = results
    print(f"{name}  Exact Match (EM): {results['exact_match']:.2f}")
    print(f"{name}  F1 Score: {results['f1']:.2f}")


Evaluating BERT...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
  4%|▍         | 43/1000 [00:00<00:16, 57.02it/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

100%|██████████| 1000/1000 [00:19<00:00, 51.11it/s]


ALBERT  Exact Match (EM): 88.30
ALBERT  F1 Score: 95.20


In [ ]:
!pip install streamlit -q
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import pipeline

st.set_page_config(page_title="Question Answering App")
st.title("📘 Question Answering with DistilBERT")

# Load DistilBERT QA pipeline
MODEL_NAME = "distilbert-base-uncased-distilled-squad"
qa_pipeline = pipeline("question-answering", model=MODEL_NAME, tokenizer=MODEL_NAME)

# Input fields
context = st.text_area("Enter the context for the question:")
question = st.text_input("Enter your question:")

# Get answer
if st.button("Get Answer"):
    if context.strip() == "" or question.strip() == "":
        st.warning("Please enter both context and question.")
    else:
        result = qa_pipeline(question=question, context=context)
        st.success(f"Answer: {result['answer']}")
        st.info(f"Confidence Score: {round(result['score'] * 100, 2)}%")


Overwriting app.py


In [ ]:
from pyngrok import ngrok
import time
import subprocess

ngrok.set_auth_token("30xQVbf58vsP56YVwbM2GHtOkkm_4XCq5cT8vz3vznACv79YN")

ngrok.kill()

subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])

time.sleep(5)

public_url = ngrok.connect(8501)
print("🔗 Streamlit app is live at:", public_url)


🔗 Streamlit app is live at: NgrokTunnel: "https://542068f6e851.ngrok-free.app" -> "http://localhost:8501"
